# Предсказание цен в Airbnb
Скриншоты карт можно посмотреть в папке "plots/airbnb"

In [44]:
import statistics
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ipyleaflet import *
from sklearn import preprocessing
 


data_file = "../csv/airbnb/air_bnb.csv"
dataset = pd.read_csv(data_file)
for feature in "id, name, host_id, host_name, last_review".split(", "):
    dataset.drop(feature,inplace=True, axis=1)
dataset.describe()



,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [70]:
dataset.sort_values(by='price',ascending=False).tail(25)

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,normalized_price
22287,Manhattan,SoHo,40.72237,-73.99817,Private room,10,5,2,0.07,1,0,-59.429385
27972,Brooklyn,Greenpoint,40.72421,-73.95364,Entire home/apt,10,1,93,4.73,1,32,-59.429385
22835,Queens,Jamaica,40.69085,-73.79916,Entire home/apt,10,1,43,1.68,1,252,-59.429385
21700,Manhattan,Harlem,40.81302,-73.95349,Shared room,10,1,0,0.00,1,0,-59.429385
34446,Bronx,Highbridge,40.83454,-73.92751,Private room,10,1,0,0.00,1,180,-59.429385
35601,Brooklyn,Bedford-Stuyvesant,40.69508,-73.95164,Entire home/apt,10,3,5,0.51,1,0,-59.429385
31066,Manhattan,Upper West Side,40.76844,-73.98333,Private room,10,1,2,0.13,1,0,-59.429385
47218,Brooklyn,Bushwick,40.69640,-73.91898,Private room,10,1,2,2.00,1,0,-59.429385
35386,Queens,Woodhaven,40.69139,-73.86086,Private room,10,7,4,0.41,2,180,-59.429385
33505,Brooklyn,Williamsburg,40.70959,-73.95693,Private room,10,1,0,0.00,1,83,-59.429385


# Посмотрим в каких колонках есть пропуски

In [6]:
dataset.isna().any()

neighbourhood_group               False
neighbourhood                     False
latitude                          False
longitude                         False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

# Заполним пропуски нулями

In [45]:
dataset['reviews_per_month'].fillna(0,inplace=True)

In [3]:
dataset.isna().any()

neighbourhood_group               False
neighbourhood                     False
latitude                          False
longitude                         False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

#  Посмотрим где расположены самые дорогие аппартаменты

In [68]:
a = Map(center=(40.7072,-73.9909), zoom = 10)
mean_price = dataset['price'].mean()
for index, row in dataset.iterrows():
    if row['price'] >= 7500:
        a.add_layer(Marker(location=(row['latitude'], row['longitude'])))
a

Map(center=[40.7072, -73.9909], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

#  Посмотрим где расположены самые дешевые аппартаменты

In [71]:
a = Map(center=(40.7072,-73.9909), zoom = 10)
mean_price = dataset['price'].mean()
for index, row in dataset.iterrows():
    if row['price'] <= 10 and row['price'] > 0:
        a.add_layer(Marker(location=(row['latitude'], row['longitude'])))
a

Map(center=[40.7072, -73.9909], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

#  Особой зависимости не видно

#  Построим Heatmap по ценам

In [61]:

a = Map(center=(40.7072,-73.9909), zoom = 10)

markers = []

dataset['normalized_price'] = preprocessing.scale(dataset['price']) * 100
for index, row in dataset.iterrows():
    markers.append([row['latitude'], row['longitude'], row['normalized_price']]) 
heatmap = Heatmap(locations=markers, radius=10)
a.add_layer(heatmap)
a

Map(center=[40.7072, -73.9909], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…